AES Encryption Setup

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import os

# Generate a 128-bit AES encryption key (16 bytes)
encryption_key = os.urandom(16)

# Generate an Initialization Vector (IV) for encryption (must be random and unique per message)
iv = os.urandom(16)


2. Encrypt Communication Data

In [ ]:
def encrypt_data(data, key, iv):
    """Encrypt data using AES-128 with CBC mode."""
    # Pad the data to make it a multiple of 128-bit (16 bytes) block size
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(data.encode()) + padder.finalize()

    # Create the AES cipher and encrypt the padded data
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    encrypted_data = encryptor.update(padded_data) + encryptor.finalize()

    return encrypted_data

# Example command to send to the drone
command = "Send GPS coordinates to Drone"

# Encrypt the command
encrypted_command = encrypt_data(command, encryption_key, iv)
print(f"Encrypted Command: {encrypted_command}")


3. Decrypt Communication Data

In [ ]:
def decrypt_data(encrypted_data, key, iv):
    """Decrypt data using AES-128 with CBC mode."""
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    # Unpad the decrypted data
    unpadder = padding.PKCS7(128).unpadder()
    original_data = unpadder.update(padded_data) + unpadder.finalize()

    return original_data.decode()

# Decrypt the encrypted command
decrypted_command = decrypt_data(encrypted_command, encryption_key, iv)
print(f"Decrypted Command: {decrypted_command}")


4. Mutual Authentication

In [ ]:
# Simulate authentication using pre-shared keys (both control station and drone know the shared key)
control_station_key = "control_station_secret_key"
drone_key = "drone_secret_key"

def authenticate(control_station_key, drone_key):
    """Simulate mutual authentication between control station and drone."""
    # Simulate authentication by comparing keys
    if control_station_key == drone_key:
        print("Authentication Successful: Communication Established")
        return True
    else:
        print("Authentication Failed: Keys Mismatch")
        return False

# Perform mutual authentication
if authenticate(control_station_key, drone_key):
    print("Secure communication can begin.")
else:
    print("Communication aborted due to authentication failure.")


5. Testing

In [ ]:
def test_communication():
    # Step 1: Authentication
    if not authenticate(control_station_key, drone_key):
        return  # Abort if authentication fails

    # Step 2: Encrypt Command from Control Station
    command = "Send GPS coordinates to Drone"
    encrypted_command = encrypt_data(command, encryption_key, iv)
    print(f"Encrypted Command: {encrypted_command}")

    # Step 3: Decrypt Command on the Drone's Side
    decrypted_command = decrypt_data(encrypted_command, encryption_key, iv)
    print(f"Decrypted Command: {decrypted_command}")

    # Verify that the decrypted command matches the original command
    assert decrypted_command == command, "Decrypted command does not match the original!"
    print("Communication successfully tested!")

# Run the test
test_communication()
